### Introduction 

Notebook explores how a LLM can call tools <br>
https://python.langchain.com/docs/how_to/tool_results_pass_to_model/
https://github.com/langchain-ai/langchain/blob/master/docs/docs/tutorials/agents.ipynb


### Setting Up Model

Using llama3.1 for quick development purposed.
Need to do ```ollama pull MODELNAME``` before running.

In [30]:
from langchain.chat_models import init_chat_model
# from langchain_tavily import TavilySearch
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.prebuilt import create_react_agent

llm = init_chat_model(
    model="ollama:llama3.1",        # or "gpt-4o-mini", "ollama/llama3.1", etc.
    temperature=0.2  # lower = more deterministic
)


### Defining Tools

In [31]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.

from langchain_core.tools import tool
#The decorator wraps in langchains type so it becomes runnable wiht .invoke(..)
@tool
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b


tools = [add, multiply]




### Testing: Can LLM see tools?

LLM can not call tools in lang chain, only request.

In [32]:
llm_with_tools = llm.bind_tools(tools) #llm_with_tools is a new wrapped llm

query = "What is 3 * 12?"

llm_with_tools.invoke(query) #



AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:21:01.838369Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4388332250, 'load_duration': 3179268417, 'prompt_eval_count': 254, 'prompt_eval_duration': 681808500, 'eval_count': 22, 'eval_duration': 525464166, 'model_name': 'llama3.1'}, id='run--ae17d5dd-0125-44d8-9e4d-e5f7a9375757-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': '272747a2-8026-442c-9b9a-af9d1521f024', 'type': 'tool_call'}], usage_metadata={'input_tokens': 254, 'output_tokens': 22, 'total_tokens': 276})

### Full Agentic Tool Use by LLM

Set up quick chat history and invoke model

In [33]:
from langchain_core.messages import HumanMessage
#message classes on lang chain inlcude human massage ,ai message, system message, and tool message
query = "What is 3 * 12? Also, what is 11 + 49?"

messages = [HumanMessage(query)]

ai_msg = llm_with_tools.invoke(messages) #llm_with_tools looks at history(currently only 1 human message) and then builds prompt
print("AI MESSAEGE CALLS")
print(ai_msg)
print("JUST AI TOOL CALLS")
print(ai_msg.tool_calls) 
#Now we add the bots message to the chat history
messages.append(ai_msg)

AI MESSAEGE CALLS
content='' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:21:03.035839Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1182342875, 'load_duration': 33382500, 'prompt_eval_count': 264, 'prompt_eval_duration': 102167584, 'eval_count': 43, 'eval_duration': 1046349958, 'model_name': 'llama3.1'} id='run--015c7f50-e679-4529-905e-eab893b63d59-0' tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'ab112ae6-2c43-4b18-a7ed-b6083a940c02', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'f37d9495-b850-4b64-8e05-56d61e9eb2db', 'type': 'tool_call'}] usage_metadata={'input_tokens': 264, 'output_tokens': 43, 'total_tokens': 307}
JUST AI TOOL CALLS
[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'ab112ae6-2c43-4b18-a7ed-b6083a940c02', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'f37d9495-b850-4b64-8e05-56d61e9eb2db', 'type': 'tool_call'}]


#### Acually call the tools and give response back

In [34]:
for tool_call in ai_msg.tool_calls: #Actually running all the tool calls ai requested in last cell
    tools_dict = {"add": add, "multiply": multiply}
    selected_tool = tools_dict[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg) #Add the tool message to chat history

messages




[HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:21:03.035839Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1182342875, 'load_duration': 33382500, 'prompt_eval_count': 264, 'prompt_eval_duration': 102167584, 'eval_count': 43, 'eval_duration': 1046349958, 'model_name': 'llama3.1'}, id='run--015c7f50-e679-4529-905e-eab893b63d59-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'ab112ae6-2c43-4b18-a7ed-b6083a940c02', 'type': 'tool_call'}, {'name': 'add', 'args': {'a': 11, 'b': 49}, 'id': 'f37d9495-b850-4b64-8e05-56d61e9eb2db', 'type': 'tool_call'}], usage_metadata={'input_tokens': 264, 'output_tokens': 43, 'total_tokens': 307}),
 ToolMessage(content='36', name='multiply', tool_call_id='ab112ae6-2c43-4b18-a7ed-b6083a940c02'),
 ToolMessage(content='60', name='add', tool_call_id='f37d9495-

#### Give final response

In [36]:
ans = llm_with_tools.invoke(messages)
ans

AIMessage(content='The result of 3 * 12 is 36. The result of 11 + 49 is 60.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-24T22:24:20.325164Z', 'done': True, 'done_reason': 'stop', 'total_duration': 739926500, 'load_duration': 42606833, 'prompt_eval_count': 130, 'prompt_eval_duration': 107827041, 'eval_count': 25, 'eval_duration': 588543667, 'model_name': 'llama3.1'}, id='run--90d7d7db-1a4b-47c0-9cc4-6a0d3dfef02e-0', usage_metadata={'input_tokens': 130, 'output_tokens': 25, 'total_tokens': 155})

#### Just the final output

In [41]:
(ans.content)

'The result of 3 * 12 is 36. The result of 11 + 49 is 60.'